# Using a CFSv2 forecast

__[CFSv2](https://data.planetos.com/datasets/ncep_cfsv2)__  is a seasonal forecast system, used for analysing past climate and also making seasonal, up to 9-month, forecasts. Here we give a brief example on how to use Planet OS API to merge 9-month forecasts started at different initial times, into a single ensemble forecast.

Ensemble forecasting is a traditional technique in medium range (up to 10 days) weather forecasts, seasonal forecasts and climate modelling. By changing initial conditions or model parameters, a range of forecasts is created, which differ from each other slightly, due to the chaotic nature of fluid dynamics (which weather modelling is a subset of). For weather forecasting, the ensemble is usually created by small changes in initial conditions, but for seasonal forecast, it is much easier to just take real initial conditions every 6-hours. Here we are going to show, first how to merge the different dates into a single plot with the help of python pandas library, and in addition we show that even 6-hour changes in initial conditions can lead to large variability in long range forecasts. 

If you have more interest in Planet OS API, please refer to our official __[documentation](data.planetos.com/datasets)__.

Please also note that the `API_client` python routine, used in this notebook, is still experimental and will change in the future, so take it just as a guidance using the API, and not as an official tool.

In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from API_client.python import datahub
from API_client.python.lib import dataset
from API_client.python.lib import variables

The API needs a file `APIKEY` with your API key in the work folder. We initialize a datahub and dataset objects.

In [2]:
dh = datahub.datahub(server='api.planetos.com',version='v1')
ds = dataset.dataset('ncep_cfsv2', dh, debug=False)

In [3]:
ds.vars=variables.variables(ds.variables(), {'reftimes':ds.reftimes,'timesteps':ds.timesteps},ds)

In order to the automatic location selection to work, add your custom location to the `API_client.python.lib.predef_locations` file.

In [17]:
for locat in ['Võru']:
    ds.vars.Convective_Precipitation_Rate_surface.get_values(count=1000, location=locat, reftime='2018-04-20T18:00:00',
                                                            reftime_end='2018-05-02T18:00:00')
    ds.vars.Maximum_temperature_height_above_ground.get_values(count=1000, location=locat, reftime='2018-04-20T18:00:00',
                                                            reftime_end='2018-05-02T18:00:00')

In [18]:
## uncomment following line to see full pandas table
## ds.vars.Convective_Precipitation_Rate_surface.values['Võru']

Here we clean the table just a bit and create time based index.

In [19]:
ddd = ds.vars.Convective_Precipitation_Rate_surface.values['Võru'][['reftime','time','Convective_Precipitation_Rate_surface']]
dd_test=ddd.set_index('time')

Next, we resample the data to 1-month totals.

In [20]:
reft_unique = ds.vars.Convective_Precipitation_Rate_surface.values['Võru']['reftime'].unique()
nf = []
for reft in reft_unique:
    abc = dd_test[dd_test.reftime==reft].resample('M').sum()
    abc['Convective_Precipitation_Rate_surface'+'_'+reft.astype(str)] = \
         abc['Convective_Precipitation_Rate_surface']*6*3600
    del abc['Convective_Precipitation_Rate_surface']
    nf.append(abc)
nf2=pd.concat(nf,axis=1)

In [21]:
# uncomment to see full pandas table
nf2

,Convective_Precipitation_Rate_surface_2019-10-20T00:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-20T06:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-20T12:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-20T18:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-21T00:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-21T06:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-21T12:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-21T18:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-22T00:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-22T06:00:00.000000000,...,Convective_Precipitation_Rate_surface_2019-10-29T18:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-30T00:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-30T06:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-30T12:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-30T18:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-31T00:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-31T06:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-31T12:00:00.000000000,Convective_Precipitation_Rate_surface_2019-10-31T18:00:00.000000000,Convective_Precipitation_Rate_surface_2019-11-01T00:00:00.000000000
time,,,,,,,,,,,,,,,,,,,,,
2019-10-31,3.5424,0.0000,0.3240,0.1728,0.5184,1.9224,0.5400,2.6352,0.0216,1.4256,...,0.0000,0.000000,0.0000,0.000000,0.0000,0.0000,0.0000,0.0000,NaN,NaN
2019-11-30,0.5400,6.0912,1.7280,0.9072,0.9288,0.5616,1.9872,0.4104,2.2680,3.6504,...,4.7088,0.496800,7.6896,0.604800,1.3824,2.8728,2.3760,3.4776,2.7648,0.1296
2019-12-31,4.1688,1.1880,3.1536,1.7280,0.9504,0.0000,0.8424,0.2160,0.8640,0.0864,...,0.1080,0.280800,3.4128,0.820800,0.0000,0.1944,0.1944,2.9160,0.1512,0.0000
2020-01-31,0.0864,0.0000,2.6568,0.0000,3.5424,0.0000,0.3888,0.9720,1.1232,0.0000,...,0.7776,2.138400,0.0000,1.317600,2.1816,2.8080,0.5832,2.2680,1.1664,0.0432
2020-02-29,0.0000,1.5336,0.0000,0.4104,3.6288,0.8856,0.0000,0.0000,1.2528,0.5832,...,0.0000,0.561600,0.4968,0.216000,0.0000,0.3672,0.0648,0.4968,1.5336,1.4040
2020-03-31,0.7344,2.6568,0.0000,4.7088,0.8208,0.0000,1.7280,0.7776,4.9464,4.6224,...,2.0088,2.613600,0.3672,9.331200,0.0000,3.5424,3.9528,0.0216,1.3392,1.6848
2020-04-30,7.4304,6.3072,7.0416,11.9232,3.4128,11.1024,0.0000,1.2312,9.8496,4.5360,...,2.0088,7.776000,2.0736,20.023200,0.1080,7.1712,4.2552,6.2424,4.1040,28.4688
2020-05-31,75.6216,16.6320,15.4872,21.0600,27.2592,19.0296,3.4776,46.5048,17.3016,12.9816,...,32.9184,39.484800,35.7264,56.332801,33.5448,36.9144,6.0048,23.4792,8.0568,5.8752
2020-06-30,15.3792,31.7952,79.7904,109.2096,49.0752,89.5752,24.9912,111.4128,3.1536,43.2648,...,108.4536,79.725600,98.9496,73.483200,30.8232,53.3952,90.1368,76.9392,82.9224,57.0024


Finally, we are visualizing the monthly precipitation for each different forecast, in a single plot. 

In [ ]:
fig=plt.figure(figsize=(10,8))
nf2.transpose().boxplot()
plt.ylabel('Monthly precipitation mm')
fig.autofmt_xdate()
plt.show()